In [1]:
import torch
from torch.utils.data import DataLoader
from torchmetrics import MeanAbsoluteError, MeanSquaredError
import numpy as np
from data.perovskite_dataset import PerovskiteDataset1d
from data.augmentations import *

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import NMF, PCA
import xgboost as xgb

# Tree based / Boosting

## Random Forest

In [6]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        rfr = RandomForestRegressor(n_estimators=100)
        rfr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = rfr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0205)
MAE: tensor(0.1029)
Fold:  1
MSE:  tensor(0.0046)
MAE: tensor(0.0504)
Fold:  2
MSE:  tensor(0.0065)
MAE: tensor(0.0571)
Fold:  3
MSE:  tensor(0.0120)
MAE: tensor(0.0709)
Fold:  4
MSE:  tensor(0.0111)
MAE: tensor(0.0770)
Mean MSE:  0.010941227
Mean MAE:  0.07164046


## Ada Boost

In [17]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        abr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5))
        abr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = abr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0204)
MAE: tensor(0.1037)
Fold:  1
MSE:  tensor(0.0050)
MAE: tensor(0.0543)
Fold:  2
MSE:  tensor(0.0065)
MAE: tensor(0.0563)
Fold:  3
MSE:  tensor(0.0117)
MAE: tensor(0.0699)
Fold:  4
MSE:  tensor(0.0105)
MAE: tensor(0.0751)
Mean MSE:  0.01084278
Mean MAE:  0.071850225


## XGBoost

In [23]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        xgbr = xgb.XGBRegressor(n_estimators = 100, tree_method="gpu_hist", n_jobs=20, max_depth=6, learning_rate=0.3,
                               booster='gbtree', num_parallel_tree=1, objective='reg:squarederror')
        xgbr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = xgbr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0239)
MAE: tensor(0.1110)
Fold:  1
MSE:  tensor(0.0046)
MAE: tensor(0.0507)
Fold:  2
MSE:  tensor(0.0087)
MAE: tensor(0.0689)
Fold:  3
MSE:  tensor(0.0119)
MAE: tensor(0.0712)
Fold:  4
MSE:  tensor(0.0119)
MAE: tensor(0.0785)
Mean MSE:  0.012195544
Mean MAE:  0.07605894


## Gradient Boosting

In [14]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        gbr = GradientBoostingRegressor()
        gbr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = gbr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0209)
MAE: tensor(0.1023)
Fold:  1
MSE:  tensor(0.0057)
MAE: tensor(0.0531)
Fold:  2
MSE:  tensor(0.0077)
MAE: tensor(0.0639)
Fold:  3
MSE:  tensor(0.0126)
MAE: tensor(0.0712)
Fold:  4
MSE:  tensor(0.0118)
MAE: tensor(0.0762)
Mean MSE:  0.011731105
Mean MAE:  0.07334508


## Decision Tree

In [9]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        dtr = DecisionTreeRegressor(max_depth=5)
        dtr.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = dtr.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0234)
MAE: tensor(0.1067)
Fold:  1
MSE:  tensor(0.0071)
MAE: tensor(0.0637)
Fold:  2
MSE:  tensor(0.0096)
MAE: tensor(0.0721)
Fold:  3
MSE:  tensor(0.0166)
MAE: tensor(0.0854)
Fold:  4
MSE:  tensor(0.0280)
MAE: tensor(0.1006)
Mean MSE:  0.016926637
Mean MAE:  0.08569111


# Clustering / Nearest Neighbors

## KNN

In [10]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        
        knn = KNeighborsRegressor(n_neighbors=20)
        knn.fit(time, label.numpy())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)#[:,3,:]
        scores = knn.predict(time)
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0221)
MAE: tensor(0.1102)
Fold:  1
MSE:  tensor(0.0047)
MAE: tensor(0.0512)
Fold:  2
MSE:  tensor(0.0104)
MAE: tensor(0.0763)
Fold:  3
MSE:  tensor(0.0151)
MAE: tensor(0.0811)
Fold:  4
MSE:  tensor(0.0136)
MAE: tensor(0.0805)
Mean MSE:  0.013186842
Mean MAE:  0.07985188


## KMeans

In [78]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(time)
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(time)
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0211)
MAE: tensor(0.1085)
Fold:  1
MSE:  tensor(0.0068)
MAE: tensor(0.0619)
Fold:  2
MSE:  tensor(0.0146)
MAE: tensor(0.0954)
Fold:  3
MSE:  tensor(0.0182)
MAE: tensor(0.0920)
Fold:  4
MSE:  tensor(0.0168)
MAE: tensor(0.0918)
Mean MSE:  0.015525341
Mean MAE:  0.08992393


## PCA + KMeans

In [14]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        pca = PCA(n_components=5)
        
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(pca.fit_transform(time))
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(pca.transform(time))
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0210)
MAE: tensor(0.1093)
Fold:  1
MSE:  tensor(0.0066)
MAE: tensor(0.0593)
Fold:  2
MSE:  tensor(0.0143)
MAE: tensor(0.0946)
Fold:  3
MSE:  tensor(0.0186)
MAE: tensor(0.0939)
Fold:  4
MSE:  tensor(0.0173)
MAE: tensor(0.0924)
Mean MSE:  0.015542507
Mean MAE:  0.08991104


## NMF + KMeans

In [13]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        nb_clusters = 10
        
        time = timeseries.numpy().reshape(-1,4*719)
        nmf = NMF(n_components=5, init='nndsvda', max_iter=600)
        
        label = label.numpy()
        
        kmeans = KMeans(nb_clusters)
        clusters = kmeans.fit_predict(nmf.fit_transform(time+2))
        
        cluster_means = []
        for i in range(nb_clusters):
            cluster_means.append(label[(clusters==i)].mean())
        
    for timeseries, label in valloader:
        
        time = timeseries.numpy().reshape(-1,4*719)
        
        val_clusters = kmeans.predict(nmf.transform(time+2))
        scores = np.array([cluster_means[i] for i in val_clusters])
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
        
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0264)
MAE: tensor(0.1211)
Fold:  1
MSE:  tensor(0.0077)
MAE: tensor(0.0683)
Fold:  2
MSE:  tensor(0.0095)
MAE: tensor(0.0703)
Fold:  3
MSE:  tensor(0.0172)
MAE: tensor(0.0913)
Fold:  4
MSE:  tensor(0.0153)
MAE: tensor(0.0861)
Mean MSE:  0.015218141
Mean MAE:  0.08742096


# Simple Mean

## Mean

In [4]:
data_dir = '/home/s522r/Desktop/perovskite/new_data/2021_KIT_PerovskiteDeposition/preprocessed'

val_mse_folds = []
val_mae_folds = []

for fold_nb in range(5):
    print('Fold: ', fold_nb)
    
    train_mean, train_std = PerovskiteDataset1d(data_dir=data_dir, transform=None, fold=fold_nb,
                                                                  split='train', label='PCE_mean').get_stats()
    
    trainset = PerovskiteDataset1d(data_dir=data_dir,
                                           transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                           split='train', label='PCE_mean', val=False)
    trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=False)
    
    valset = PerovskiteDataset1d(data_dir=data_dir,
                                         transform=normalize_1d(train_mean, train_std), fold=fold_nb,
                                         split='train', label='PCE_mean', val=True)
    valloader = DataLoader(valset, batch_size=len(valset), shuffle=False)
    
    for timeseries, label in trainloader:
        
        mean = label.numpy().mean()
        
    for timeseries, label in valloader:
        
        scores = np.array([mean]*len(label))
        
        mse = MeanSquaredError()
        mae = MeanAbsoluteError()
        
        val_mse = mse(torch.from_numpy(scores), label)
        print('MSE: ',val_mse)
        val_mse_folds.append(val_mse)
        
        val_mae = mae(torch.from_numpy(scores), label)
        print('MAE:',val_mae)
        val_mae_folds.append(val_mae)
    
    
print('Mean MSE: ', np.mean(val_mse_folds))
print('Mean MAE: ', np.mean(val_mae_folds))

Fold:  0
MSE:  tensor(0.0210)
MAE: tensor(0.1095)
Fold:  1
MSE:  tensor(0.0107)
MAE: tensor(0.0830)
Fold:  2
MSE:  tensor(0.0129)
MAE: tensor(0.0854)
Fold:  3
MSE:  tensor(0.0203)
MAE: tensor(0.1006)
Fold:  4
MSE:  tensor(0.0192)
MAE: tensor(0.0957)
Mean MSE:  0.016806483
Mean MAE:  0.09483276
